In [1]:
import pandas as pd
import os
import numpy as np
import proplot as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
os.getcwd()

'D:\\Py_works\\00Dh_process'

In [2]:
dxt_index_pth="D:\\Key_works\\20231212dh_calculate\\0data\\0shp\\dxt_indext_prj.shp"
dxt_df=gpd.read_file(dxt_index_pth)
dxt_df.head(1)


,OBJECTID,TOPODEM,TOPOMAP,Scale,AcquirData,topodemID,topodemID1,OLDM,NEWM,图号,比例尺,制图时,制图_1,说明,data,region,sign,flag,yr_range,geometry
0,2,g46e003010.tif,None,10,0,g46e003010,G46E003010,None,None,None,None,1980年11月航摄,None,None,1980,14EH,1980,0001_1980,1980-1984,"POLYGON ((-1244748.017 2990507.262, -1245753.7..."


In [4]:
pth='D:/Key_works/20231212dh_calculate/xnhl_test'
df_log=pd.read_csv(pth+'/dh-effe-stat.csv')
df_log.head(1)

,Unnamed: 0,group_id,mean,median,mean_fill,median_fill,Glc_Area,Mean_Elev,Mean_Slp,Mean_Asp,MA_Elev,basin
0,G101096E33298N,g0018,-2.247736,-4.501035,-2.532606,-4.124494,293031.2539,4942.3,39.4,315.8,4939.9,Yellow


In [5]:
df_grp=pd.read_csv('D:/Key_works/20231212dh_calculate/0data/dxt_group.csv',index_col=0)
df_grp.head(1)

,g0068,g0377,g0001,g0072,g0381,g0253,g0350,g0518,g0063,g0497,...,g0428,g0194,g0564,g0344,g0538,g0277,g0401,g0078,g0282,g0444
0,j47e006009,j45e019011,i48e021008,j47e005012,j45e016014,h45e012016,i44e020003,j43e005007,j47e010012,j43e017010,...,i44e016005,h47e016005,l44e019008,g45e003021,k45e007002,h45e010003,j46e020001,j47e006007,h45e012001,i44e009010


In [6]:
stat_df=list(df_log.iloc[:,0])

In [7]:
len(stat_df[0]),stat_df[0]

(14, 'G101096E33298N')

In [8]:
pth_nc='D:\\Key_works\\20231212dh_calculate\\xnhl_test\\gla_level_dh_final'

In [10]:
import glob
newp=pth_nc+'\\nc_yr'
if os.path.exists(newp):pass
else:os.makedirs(newp)

for glai in stat_df: #G075167E36993N stat_df
    outnm=newp+'/'+glai+'.nc'
    if os.path.exists(outnm):
        continue
    ########
    gla=glob.glob(os.path.join(pth_nc+'/nc','*'+glai+'.nc'))[0]
    ds_gla=xr.open_dataset(gla)
    #basenm=os.path.basename(gla)
    ######获取group内地形图索引号
    #print(gpi)
    gpi=ds_gla.attrs['topo process group']
    dxt_lst=list(df_grp[gpi].dropna())
    #########
    ttt=[]#存储裁剪结果
    yyy=[]#存储地形图年份
    ds_gla['dxt_id']=(('y','x'),ds_gla['elevation'].values*0+1)#创建一个地形图年份索引图层
    
    
    for flag in dxt_lst: #['0617_1957', '0612_1963','0613_1963']  dxt_df.flag
        dxti=dxt_df[dxt_df.topodemID==flag]
        try:
            ds_gla_t=ds_gla.rio.write_crs(ds_gla.crs)
            gla_index=ds_gla_t.rio.clip(dxti.geometry)
            yyy.append(dxti.sign.values[0])
            ttt.append(gla_index)
            #print('yes')
        except:
            #print('no')
            pass   
    ########################################
    if len(ttt)>1: #多于两个说明地形图进行了裁剪，此时需要分别赋予年份
        for j,ti in enumerate(ttt):
            pp=ti['topo_dh'].to_dataframe().dropna().reset_index()
            for k,yi in enumerate(pp.y):
                xi=pp.x[k]
                ds_gla['dxt_id'].loc[(yi,xi)]=yyy[j]     
    elif len(ttt)==1:
        ds_gla['dxt_id']=ds_gla['dxt_id']*yyy[0]
    else:
        continue
    ########################################################################

    ########################################################################
    ####存储#####
    ds_gla.to_netcdf(outnm)
    ###########################################    
    #break
print('done!')

done!
